In [ ]:
import getpass
from arcgis.gis import *

password = getpass.getpass("Please enter password: ")
dev_gis = GIS('https://www.arcgis.com', 'johnnyDev', password)
print("Successfully logged in to {} as {}".format(dev_gis.properties.urlKey + '.' + dev_gis.properties.customBaseUrl,
                                                 dev_gis.users.me.username))

In [ ]:
flyr_search_results = dev_gis.content.search(query="""title: "Griffith*" AND 
                                         type: "Feature Service" """, 
                                         max_items=10)
flyr_search_results

In [ ]:
flyr_collection_item = flyr_search_results[0]
flayers = flyr_collection_item.layers
flayer = flayers[0]
flayer.properties.name

In [ ]:
from arcgis import geometry
from arcgis import features

def create_feature(map1, g):
    try:
        oid = 1
        pt = geometry.Point(g)
        feat = features.Feature(geometry=pt, attributes={'OBJECTID': oid})
        flayer.edit_features(adds=[feat])
        print(str(g))
        map1.draw(g)
        oid += 1
    except:
        print("Couldn't create the feature. Try again, please...")

In [ ]:
map1 = dev_gis.map('Los Angeles', 10)
map1

In [ ]:
map1.on_click(create_feature)

In [ ]:
map1.clear_graphics()

In [ ]:
map1.add_layer(flayer)